In [1]:
import pandas as pd
import pca
import numpy as np
D=pd.read_excel("https://data.hossam.kr/E04/diamonds.xlsx")
data=pd.DataFrame(D)
clarity_rank='FL>IF>VVS1>VVS2>VS1>VS2>SI1>SI2>I1>I2>I3'
clarity_rank=clarity_rank.split(">")
clarity_rank={y:len(clarity_rank)-x for x,y in enumerate(clarity_rank)}
numeric=['carat','x','y','z','depth','table']
target="price"
category={'cut',"clarity",'color'}

----------

요인적재값(Factor Loadings)은 주성분 분석(PCA) 또는 요인 분석(Factor Analysis)의 결과로 얻는 값으로, 주성분(요인)과 원래 변수 간의 관계를 나타내는 계수입니다. 요인적재값은 각 변수가 주성분(요인)에 대해 어떻게 설명되는지를 보여줍니다.

주성분 분석에서 요인적재값을 얻는 수식은 다음과 같습니다:

$ X = FL + E $

여기서:

- $X$는 $n \times p$ 크기의 데이터 행렬로, $n$은 관측값(샘플)의 수, $p$은 변수(특성)의 수를 나타냅니다.
- $F$는 $n \times k$ 크기의 주성분(요인) 행렬로, $k$는 추출한 주성분(요인)의 개수를 나타냅니다.
- $L$은 $k \times p$ 크기의 요인적재값 행렬입니다.
- $E$는 잔차 행렬로, 모델로 설명되지 않는 오차를 나타냅니다.

주성분 분석을 통해 $F$와 $L$을 추정한 후, $L$ 행렬의 각 원소는 변수와 주성분(요인) 간의 관계를 나타내는데 사용됩니다. 이러한 관계는 주로 상관 계수나 회귀 계수의 형태로 표현됩니다.

요인 분석의 경우에도 주성분 분석과 유사한 수식을 사용하여 요인적재값을 계산하며, 이것은 요인과 변수 간의 관계를 나타냅니다. 요인 분석에서는 주로 회귀 계수와 요인의 곱으로 나타낼 수 있습니다.


-----------

어렵게 생각할 필요가 없다.          
PCA의 예시를 그대로 가져와서 보자

## PCA

In [4]:
numer_data=data[numeric]
corr=numer_data.corr(method="pearson")

In [15]:
import numpy as np

* 이전에 보았지만 2번쨰 값들이 eigen_vector들이며, 이들이 바로 우리가 원하던 $L$의 (의미상)역행렬이다

In [17]:
eig=np.linalg.eig(corr)
eig

(array([3.93213863, 1.28405203, 0.68339837, 0.0132108 , 0.04740003,
        0.03980014]),
 array([[-4.95367277e-01, -4.51296690e-02,  2.79083244e-02,
          3.19502171e-01, -7.89965363e-01, -1.60214816e-01],
        [-5.00910150e-01, -8.20368481e-03,  6.99792387e-02,
         -8.60289644e-01, -4.07590290e-02,  4.86318467e-02],
        [-4.95217561e-01, -9.65736681e-03,  8.62272230e-02,
          2.34074490e-01,  5.37622283e-01, -6.35139814e-01],
        [-4.93882084e-01, -1.01283089e-01, -7.50890524e-03,
          3.16449645e-01,  2.91337943e-01,  7.48830865e-01],
        [ 6.82243940e-04, -7.34082087e-01, -6.71000723e-01,
         -5.36383790e-02,  1.40298568e-02, -8.83580268e-02],
        [-1.20581388e-01,  6.69826823e-01, -7.32523408e-01,
          3.43081153e-03,  1.34563727e-02, -2.96106573e-03]]))

* 요인을 2개로 추출했었음

In [34]:
re_L=pd.DataFrame(eig[1][:2]).T.rename(columns=dict(zip([0,1],["제1주성분","제2주성분"]))).rename(dict(zip(range(6),numeric)))
re_L

,제1주성분,제2주성분
carat,-0.495367,-0.500910
x,-0.045130,-0.008204
y,0.027908,0.069979
z,0.319502,-0.860290
depth,-0.789965,-0.040759
table,-0.160215,0.048632


위의 행렬이 feature들을 주성분 1,2로 변환하는 행렬이다          
아래는 1,2 주성분의 값이다 

In [40]:
partial=numer_data@re_L
partial

,제1주성분,제2주성분
0,-56.799416,-1.791529
1,-56.447396,-1.326500
2,-54.808065,-1.009007
3,-57.961156,-1.868985
4,-58.646661,-2.011672
...,...,...
53935,-56.499289,-2.721894
53936,-57.958206,-3.007742
53937,-58.528958,-2.703975
53938,-56.818173,-2.936099


- 이제 이를 원래의 주성분으로 돌리는 행렬을 구해야한다
- 이는 최소제곱법으로 근사해를 구해야한다

In [53]:
import scipy as sp

In [92]:
L,E=sp.linalg.lstsq(partial,numer_data)[0],sp.linalg.lstsq(partial,numer_data)[1]

In [93]:
pd.DataFrame(L).T.abs().rename(columns=dict(zip([0,1],["제1주성분","제2주성분"]))).rename(dict(zip(range(6),numeric))).style.background_gradient(cmap="PuBu")

,제1주성분,제2주성분
carat,0.015187,0.593226
x,0.031641,1.391252
y,0.032148,1.382049
z,0.017507,0.900880
depth,1.076110,0.014077
table,1.014240,0.258202


아래에서 비교해보라

In [117]:
numer_data

,carat,x,y,z,depth,table
0,0.23,3.95,3.98,2.43,61.5,55.0
1,0.21,3.89,3.84,2.31,59.8,61.0
2,0.23,4.05,4.07,2.31,56.9,65.0
3,0.29,4.20,4.23,2.63,62.4,58.0
4,0.31,4.34,4.35,2.75,63.3,58.0
...,...,...,...,...,...,...
53935,0.72,5.75,5.76,3.50,60.8,57.0
53936,0.72,5.69,5.75,3.61,63.1,55.0
53937,0.70,5.66,5.68,3.56,62.8,60.0
53938,0.86,6.15,6.12,3.74,61.0,58.0


In [115]:
partial@sp.linalg.lstsq(partial,numer_data)[0]

,0,1,2,3,4,5
0,0.200149,4.289664,4.301955,2.608318,61.147644,57.145653
1,-0.070373,3.631552,3.647945,2.183220,60.762286,56.908692
2,-0.233820,3.137969,3.156453,1.868497,58.993716,55.327995
3,0.228454,4.434184,4.446350,2.698435,62.398895,58.303937
4,0.302689,4.654388,4.665588,2.838980,63.138583,58.962361
...,...,...,...,...,...,...
53935,0.756623,5.574539,5.578116,3.441211,60.837772,56.601030
53936,0.904038,6.018389,6.020074,3.724267,62.411751,58.006915
53937,0.715168,5.613831,5.618601,3.460601,63.021666,58.664228
53938,0.878852,5.882644,5.884410,3.639767,61.183941,56.869147


#### 책에서 설명한 방법(상관계수 이용)

In [70]:
factor_analysis=pd.merge(numer_data,partial,right_index=True,left_index=True).corr(method='pearson')[partial.columns]

In [81]:
factor_analysis.iloc[:-2].abs().style.background_gradient(cmap="PuBu")

,제1주성분,제2주성분
carat,0.126666,0.950397
x,0.066158,0.948338
y,0.054898,0.924711
z,0.166591,0.982115
depth,0.942482,0.204073
table,0.033483,0.005299


#### 이래서 MinMax scale을 했구나

standard_scale을 통한 변수설명은 위의 해석을 어렵게 함.         
즉, 하나하나의 값의 범위가 일정하지 않으면 요인적재값에서 얼마나 많이 변동하는지가 해석하기 어려워짐

> 이럴땐 상관계수를 이용한 해석이 더 용이함